In [ ]:
import torch
from autoemulate.experimental.emulators import GaussianProcessExact
from autoemulate.experimental.emulators.random_forest import RandomForest
from autoemulate.experimental.emulators.transformed.base import TransformedEmulator
from autoemulate.experimental.transforms import PCATransform, VAETransform
from sklearn.datasets import make_regression
import torchmetrics
from autoemulate.experimental.model_selection import evaluate
from autoemulate.experimental.types import TensorLike

# Uncomment to enable logging for GPs
import logging
# logging.basicConfig(level=logging.INFO)


def make_data(
        random_state: int = 42,
        n_samples: int = 200,
        n_informative:int = 2,
        n_features: int = 5,
        noise: float = 0.2,
        n_targets: int = 2
    ):
    x, y, _ = make_regression(
        n_samples=n_samples,
        n_features=n_features,
        noise=noise,
        random_state=random_state,
        n_informative=n_informative,
        n_targets=n_targets,
        coef=True,
    )
    x = torch.tensor(x, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)
    return x, y

# Train data
x, y = make_data(random_state=42, n_targets=2)

# Test data
x2, y2 = make_data(random_state=43)

# Make transform
pca = PCATransform(n_components=2)
pca.fit(x)

# Transform data
print(pca(x).shape)

# Invert PCA on tensor
print(pca.inv(pca(x)).shape)

In [ ]:
# Example y transform
pca_y = PCATransform(n_components=1)
pca_y.fit(y)
print(pca_y(y)[:10])
print(pca_y.inv(pca_y(y))[:10])



In [ ]:
# Example tranform GP output from PCA space to original space
pca_y = PCATransform(n_components=1)
pca_y.fit(y)
gp = GaussianProcessExact(x, pca_y(y), epochs=50)
gp.fit(x, pca_y(y))
pred = gp.predict(x[:10])
print(pred)
print(pca_y._inverse_gaussian(pred))

In [ ]:
# Example using transformed emulator with GP and Random Forest
for model in [GaussianProcessExact, RandomForest]:
    # Create transformed emulator with GP
    emulator = TransformedEmulator(
        x=x,
        y=y,
        transforms=[PCATransform(n_components=4), VAETransform(latent_dim=2)],
        target_transforms=[PCATransform(n_components=1)],
        model=model,
        epochs=100,
    )

    # Fit
    emulator.fit(x, y)

    # Predict on training data
    pred = emulator.predict(x[:30])
    pred = pred if isinstance(pred, TensorLike) else pred.mean
    print(f"Train R2: {evaluate(y[:30], pred, torchmetrics.R2Score, device=pred.device):.3f}")

    # Predict on test data
    pred = emulator.predict(x2)
    pred = pred if isinstance(pred, TensorLike) else pred.mean
    print(f"Test  R2: {evaluate(y2, pred, torchmetrics.R2Score, device=pred.device):.3f}",)